In [ ]:
import pandas as pd
import numpy as np
import nltk
import spacy
import re
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)

In [ ]:
df_question = pd.read_hdf("/content/drive/MyDrive/auto_tagging_data_v2.h5")


In [ ]:
df_question.head(10)

,Id,Title,Body,Tags
0,6,The Two Cultures: statistics vs. machine learning?,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach...",[machine-learning]
1,21,Forecasting demographic census,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...,[forecasting]
2,22,Bayesian and frequentist reasoning in plain English,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n,[bayesian]
3,31,What is the meaning of p values and t values in statistical tests?,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests....","[hypothesis-testing, t-test, p-value, interpretation]"
4,36,Examples for teaching: Correlation does not mean causation,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ...",[correlation]
5,93,Robust nonparametric estimation of hazard/survival functions based on low count data,"<p>We're trying to use a Gaussian process to model h(t) -- the hazard function -- for a very small initial population, and then fit that using the available data. While this gives us nice plots f...","[nonparametric, survival]"
6,95,How Large a Difference Can Be Expected Between Standard GARCH and Asymmetric GARCH Volatility Forecasts?,<p>I have been using various GARCH-based models to forecast volatility for various North American equities using historical daily data as inputs.</p>\n\n<p>Asymmetric GARCH models are often cited ...,[time-series]
7,103,What is your favorite data visualization blog?,<p>What is the best blog on data visualization?</p>\n\n<p>I'm making this question a community wiki since it is highly subjective. Please limit each answer to one link.</p>\n\n<hr>\n\n<p><strong>...,"[data-visualization, references]"
8,113,What are some good frameworks for method selection?,"<p>I have been looking into theoretical frameworks for method selection (note: not model selection) and have found very little systematic, mathematically-motivated work. By 'method selection', I m...",[machine-learning]
9,114,What statistical blogs would you recommend?,"<p>What statistical research blogs would you recommend, and why?</p>\n",[references]


In [ ]:
df_question.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76365 entries, 0 to 76364
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      76365 non-null  int64 
 1   Title   76365 non-null  object
 2   Body    76365 non-null  object
 3   Tags    76365 non-null  object
 4   Text    76365 non-null  object
dtypes: int64(1), object(4)
memory usage: 5.5+ MB


In [ ]:
df_question['Text'] = df_question["Title"] + " " + df_question["Body"]


In [ ]:
df_question.head(5)

,Id,Title,Body,Tags,Text
0,6,The Two Cultures: statistics vs. machine learning?,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach...",[machine-learning],the two cultures statistics vs machine learning last year i read a blog post from brendan o connor entitled statistics vs machine learning fight that discussed some of the differences between the ...
1,21,Forecasting demographic census,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...,[forecasting],forecasting demographic census what are some of the ways to forecast demographic census with some validation and calibration techniques some of the concerns census blocks vary in sizes as rural ar...
2,22,Bayesian and frequentist reasoning in plain English,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n,[bayesian],bayesian and frequentist reasoning in plain english how would you describe in plain english the characteristics that distinguish bayesian from frequentist reasoning
3,31,What is the meaning of p values and t values in statistical tests?,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests....","[hypothesis-testing, t-test, p-value, interpretation]",what is the meaning of p values and t values in statistical tests after taking a statistics course and then trying to help fellow students i noticed one subject that inspires much head desk bangin...
4,36,Examples for teaching: Correlation does not mean causation,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ...",[correlation],examples for teaching correlation does not mean causation there is an old saying correlation does not mean causation when i teach i tend to use the following standard examples to illustrate this p...


In [ ]:
def clean_text(text):
    # remove html tags and url links
    text = re.sub(r'<.*?>', '', text)
    # remove everything alphabets
    text = re.sub("[^a-zA-Z]"," ",text)
    # remove whitespaces
    text = ' '.join(text.split())

    return text

In [ ]:
df_question['Text'] = df_question['Text'].apply(lambda x: clean_text(x))
df_question['Text'] = df_question['Text'].str.lower()

In [ ]:
df_question[['Id', 'Text', 'Tags']].sample(5)


,Id,Text,Tags
11893,83834,poor parameter estimation in frailty models i fitted a one parameter gamma shared frailty model with weibull and exponential baseline hazards for recurrent event data the dataset contained subject...,[survival]
68279,176329,calculate the probability of observing n elements in m draws from an uneven frequency distribution in order but consecutive or not i want to calculate the probability to observe a certain sequence...,[probability]
34117,115126,pooling imputed still not analysed datasets in mice i need to do a multiple imputation on a dataset with several missing values and i need to do it with mice because later i ll have to compare the...,"[r, regression]"
39674,140442,interpretation p value one sample ks test i have some difficulties with understanding the output of the ks test in r suppose i want to test whether my data follows a exponential distribution with ...,"[p-value, goodness-of-fit]"
67938,208337,how can i combine nominal with ordinal data to build a unique variable i performed an interview with questions protocol the structure of questions is based on variables major variables are coming ...,"[correlation, categorical-data, data-transformation, ordinal]"


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_question['Text'])

In [ ]:
len(tokenizer.word_index)


81956

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

81957

In [ ]:
sequences = tokenizer.texts_to_sequences(df_question['Text'])


In [ ]:
i = 0
print(df_question['Text'][i], '\n'), print(sequences[i])

the two cultures statistics vs machine learning last year i read a blog post from brendan o connor entitled statistics vs machine learning fight that discussed some of the differences between the two fields andrew gelman responded favorably to this simon blomberg from r s fortunes package to paraphrase provocatively machine learning is statistics minus any checking of models and assumptions brian d ripley about the difference between machine learning and statistics user vienna may season s greetings andrew gelman in that case maybe we should get rid of checking of models and assumptions more often then maybe we d be able to solve some of the problems that the machine learning people can solve but we can t there was also the statistical modeling the two cultures paper by leo breiman in which argued that statisticians rely too heavily on data modeling and that machine learning techniques are making progress by instead relying on the predictive accuracy of models has the statistics field 

(None, None)

In [ ]:
seq_lengths = []

for i in sequences:
    seq_lengths.append(len(i))

In [ ]:
print("30th percentile: ", pd.Series(seq_lengths).quantile(0.3))
print("40th percentile: ", pd.Series(seq_lengths).quantile(0.4))
print("50th percentile: ", pd.Series(seq_lengths).quantile(0.5))
print("60th percentile: ", pd.Series(seq_lengths).quantile(0.6))
print("70th percentile: ", pd.Series(seq_lengths).quantile(0.7))
print("80th percentile: ", pd.Series(seq_lengths).quantile(0.8))
print("90th percentile: ", pd.Series(seq_lengths).quantile(0.9))
print("95th percentile: ", pd.Series(seq_lengths).quantile(0.95))
print("99th percentile: ", pd.Series(seq_lengths).quantile(0.99))

30th percentile:  97.0
40th percentile:  116.0
50th percentile:  137.0
60th percentile:  162.0
70th percentile:  193.0
80th percentile:  238.0
90th percentile:  320.0
95th percentile:  411.0
99th percentile:  678.0


In [ ]:
max_length = 125

# padding
padded_seq = pad_sequences(sequences, maxlen=max_length)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_question['Tags'])
y = multilabel_binarizer.transform(df_question['Tags'])

In [ ]:
padded_seq.shape, y.shape


((76365, 125), (76365, 100))

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(padded_seq, y, test_size=0.2, random_state=9)

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size+1, 128, input_length = max_length))
model.add(Dropout(0.15))
model.add(Conv1D(300, 3, padding = 'valid', activation = "relu", strides = 1))
model.add(GlobalMaxPool1D())
model.add(Dense(100, activation = "sigmoid"))
#model.add(Activation('sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 125, 128)          10490624  
                                                                 
 dropout (Dropout)           (None, 125, 128)          0         
                                                                 
 conv1d (Conv1D)             (None, 123, 300)          115500    
                                                                 
 global_max_pooling1d (Glob  (None, 300)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 100)               30100     
                                                                 
Total params: 10636224 (40.57 MB)
Trainable params: 10636224 (40.57 MB)
Non-trainable params: 0 (0.00 Byte)
______________

In [ ]:
callbacks = [EarlyStopping(patience=3),ModelCheckpoint(filepath='model-conv1d_v1.h5', save_best_only=True)]


In [ ]:
history = model.fit(x_train, y_train,epochs=10,batch_size=512,validation_split=0.1,callbacks=callbacks)


Epoch 1/10
108/108 [==============================] - ETA: 0s - loss: 0.1896 - accuracy: 0.0365

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


108/108 [==============================] - 154s 1s/step - loss: 0.1896 - accuracy: 0.0365 - val_loss: 0.0906 - val_accuracy: 0.0592
Epoch 2/10
108/108 [==============================] - 139s 1s/step - loss: 0.0895 - accuracy: 0.0621 - val_loss: 0.0878 - val_accuracy: 0.0812
Epoch 3/10
108/108 [==============================] - 141s 1s/step - loss: 0.0840 - accuracy: 0.1265 - val_loss: 0.0797 - val_accuracy: 0.1756
Epoch 4/10
108/108 [==============================] - 142s 1s/step - loss: 0.0749 - accuracy: 0.2087 - val_loss: 0.0711 - val_accuracy: 0.2316
Epoch 5/10
108/108 [==============================] - 143s 1s/step - loss: 0.0673 - accuracy: 0.2540 - val_loss: 0.0652 - val_accuracy: 0.2712
Epoch 6/10
108/108 [==============================] - 135s 1s/step - loss: 0.0620 - accuracy: 0.2931 - val_loss: 0.0613 - val_accuracy: 0.2939
Epoch 7/10
108/108 [==============================] - 134s 1s/step - loss: 0.0584 - accuracy: 0.3201 - val_loss: 0.0589 - val_accuracy: 0.3188
Epoch 8/10

In [ ]:
preds = model.predict(x_val)


478/478 [==============================] - 15s 32ms/step


In [ ]:
preds.shape


(15273, 100)

In [ ]:
preds_int = (preds >= 0.45).astype(int)



In [ ]:
from sklearn.metrics import f1_score

# calculate F1 score
f1_score(y_val, preds_int, average="micro")


0.4762877104478209

In [ ]:
def infer_tags(q):
    q = clean_text(q)
    q = q.lower()
    q_seq = tokenizer.texts_to_sequences([q])
    q_seq_padded = pad_sequences(q_seq, maxlen=300)
    q_pred = model.predict(q_seq_padded)
    q_pred = (q_pred >= 0.3).astype(int)
    return multilabel_binarizer.inverse_transform(q_pred)

In [ ]:
new_q = "Regression line in ggplot doesn't match computed regression Im using R and created a chart using ggplot2. I then create a regression so I can make some predicitions I pass my data frame of to the predict function predict(regression, Measures) I'd expect the predictions to be the same as if I used the regression line on the chart, but they aren't the same. Why would this be the case? Is there a setting in ggplot or is my expectation incorrect?"
newq= "Including -Inf on ggplot2 axis"

infer_tags(newq)

1/1 [==============================] - 0s 62ms/step


[('data-visualization', 'r')]

In [ ]:
 new_q="What is the difference between supervised learning and unsupervised learning?"
 infer_tags(new_q)

1/1 [==============================] - 0s 34ms/step


[('machine-learning',)]